# Contenedores en Desarrollo

## Problemas

* Línas de comando muy largas `docker run .....`
* Si cada contenedor corre un servicio, tenemos que levantar tantos contenedores como componentes tenga nuestra aplicación, y luego dentenerlos... 😫
* Definición de redes manuales
* Gestión de nombres
* etc.

# [Docker Compose](https://docs.docker.com/compose/overview/)

<img src="images/docker-compose-logo.png" width="200px">

1. Docker compose resuelve el problema de ejecutar **mútiples contenedores**, utilizando un archivo de definición de configuración [YAML](http://yaml.org/) (puede usar JSON u otros).
2. Utiliza comandos más simples:
    ```
    docker-compose up
    
    ```
3. Casi todas las opiciones de docker run están disponibles desde docker-compose.


Veamos un ejemplo...

In [4]:
%%bash
mkdir -p compose_wordpress

In [5]:
%%writefile compose_wordpress/docker-compose.yml

version: '3.1'

services:

  wordpress:
    image: wordpress
    ports:
      - 8080:80
    environment:
      WORDPRESS_DB_PASSWORD: example

  mysql:
    image: mysql:5.7
    environment:
      MYSQL_ROOT_PASSWORD: example


Writing compose_wordpress/docker-compose.yml


In [10]:
!cd compose_wordpress && docker-compose up

Pulling wordpress (wordpress:latest)...
latest: Pulling from library/wordpress

ad1a0d33: Already exists
f85af454: Pulling fs layer
42e7826b: Pulling fs layer
d91a9039: Pulling fs layer
a95ab677: Pulling fs layer
8373e193: Pulling fs layer
2b2280ed: Pulling fs layer
b84cbbbc: Pulling fs layer
3c21f7f1: Pulling fs layer
12209ba4: Pulling fs layer
c49c13da: Pulling fs layer
cc3dbc7d: Pulling fs layer
e37f04ae: Pulling fs layer
cfb9a6c6: Pulling fs layer
c5fef428: Pulling fs layer
a3665466: Pulling fs layer
b117fcc0: Pulling fs layer
Digest: sha256:0be1858d894630ab85b747a4321045a4825c40cbfe5aa3d11248334a71888e2b 3.221kB/3.221kBB
Status: Downloaded newer image for wordpress:latest
Pulling mysql (mysql:5.7)...
5.7: Pulling from library/mysql

ad1a0d33: Already exists
d83dece3: Pulling fs layer
ce7b50d3: Pulling fs layer
0a3a64e4: Pulling fs layer
6a82c85b: Pulling fs layer
6f1a1c40: Pulling fs layer
414eac71: Pulling fs layer
28654a91: Pulling fs layer
93eb988c: Pulling fs layer
c3585729: P

mysql_1      | 2017-09-20T14:09:38.489139Z 0 [Note] InnoDB: Completed initialization of buffer pool
mysql_1      | 2017-09-20T14:09:38.502687Z 0 [Note] InnoDB: If the mysqld execution user is authorized, page cleaner thread priority can be changed. See the man page of setpriority().
mysql_1      | 2017-09-20T14:09:38.527782Z 0 [Note] InnoDB: Highest supported file format is Barracuda.
mysql_1      | 2017-09-20T14:09:38.563506Z 0 [Note] InnoDB: Creating shared tablespace for temporary tables
mysql_1      | 2017-09-20T14:09:38.563775Z 0 [Note] InnoDB: Setting file './ibtmp1' size to 12 MB. Physically writing the file full; Please wait ...
mysql_1      | 2017-09-20T14:09:39.009383Z 0 [Note] InnoDB: File './ibtmp1' size is now 12 MB.
mysql_1      | 2017-09-20T14:09:39.011792Z 0 [Note] InnoDB: 96 redo rollback segment(s) found. 96 redo rollback segment(s) are active.
mysql_1      | 2017-09-20T14:09:39.011909Z 0 [Note] InnoDB: 32 non-redo rollback segment(s) are active.
mysql_1      | 2017-0

mysql_1      | 2017-09-20T14:09:43.848038Z 0 [Note] InnoDB: Dumping buffer pool(s) to /var/lib/mysql/ib_buffer_pool
mysql_1      | 2017-09-20T14:09:43.848789Z 0 [Note] InnoDB: Buffer pool(s) dump completed at 170920 14:09:43
wordpress_1  | 
wordpress_1  | MySQL Connection Error: (2002) Connection refused
wordpress_1  | 
wordpress_1  | Warning: mysqli::mysqli(): (HY000/2002): Connection refused in - on line 22
mysql_1      | 2017-09-20T14:09:45.746542Z 0 [Note] InnoDB: Shutdown completed; log sequence number 12144294
mysql_1      | 2017-09-20T14:09:45.752000Z 0 [Note] InnoDB: Removed temporary tablespace data file: "ibtmp1"
mysql_1      | 2017-09-20T14:09:45.752113Z 0 [Note] Shutting down plugin 'CSV'
mysql_1      | 2017-09-20T14:09:45.752140Z 0 [Note] Shutting down plugin 'MEMORY'
mysql_1      | 2017-09-20T14:09:45.752154Z 0 [Note] Shutting down plugin 'PERFORMANCE_SCHEMA'
mysql_1      | 2017-09-20T14:09:45.752529Z 0 [Note] Shutting down plugin 'sha256_password'
mysql_1      | 2017-09-

wordpress_1  | 172.27.0.1 - - [20/Sep/2017:14:13:39 +0000] "GET /wp-includes/js/jquery/jquery.js?ver=1.12.4 HTTP/1.1" 200 34120 "http://localhost:8080/wp-admin/install.php" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36"
wordpress_1  | 172.27.0.1 - - [20/Sep/2017:14:13:39 +0000] "GET /wp-includes/js/jquery/jquery-migrate.min.js?ver=1.4.1 HTTP/1.1" 200 4366 "http://localhost:8080/wp-admin/install.php" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36"
wordpress_1  | 172.27.0.1 - - [20/Sep/2017:14:13:39 +0000] "GET /wp-admin/js/language-chooser.min.js?ver=4.8.1 HTTP/1.1" 200 590 "http://localhost:8080/wp-admin/install.php" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36"
wordpress_1  | 172.27.0.1 - - [20/Sep/2017:14:13:39 +0000] "GET /wp-admin/images/spinner-2x.gif HT

# Accediendo a http://localhost:8080
![](images/wp.png)

# Qué está sucediendo?
<img src="images/dibujo.svg">

# ¿Que sucedió?
* Se crearon las imágenes:

    `docker images`
* Se crearon los contenedores:

    `docker ps`
* Se creó una red
* Se expusieron los puertos


# Un ejmplo un [poco más completo](https://docs.docker.com/compose/wordpress/#define-the-project)

```yaml
version: '3'

services:
   db:
     image: mysql:5.7
     volumes:
       - db_data:/var/lib/mysql
     restart: always
     environment:
       MYSQL_ROOT_PASSWORD: somewordpress
       MYSQL_DATABASE: wordpress
       MYSQL_USER: wordpress
       MYSQL_PASSWORD: wordpress

   wordpress:
     depends_on:
       - db
     image: wordpress:latest
     ports:
       - "8000:80"
     restart: always
     environment:
       WORDPRESS_DB_HOST: db:3306
       WORDPRESS_DB_USER: wordpress
       WORDPRESS_DB_PASSWORD: wordpress
volumes:
    db_data:
```

# Desarrollando con Docker
* Muchos frameworks tienen servidores que *recompilan* nuestro códigos con los cambios que hacemos.
* Para esto es útil utilizar un VOLUME para comparir el código fuente.
* Este enfoque requiere tener dos archivos docker-compose.yml:
    * uno para desarrollo con el VOLUME
    * uno para producción con un COPY

## Ejemplo

In [35]:
%%bash
mkdir -p compose_flask
mkdir -p compose_flask/code

In [55]:
%%writefile compose_flask/code/app.py

from flask import Flask
app = Flask(__name__)
@app.route("/")
def index():
    return "Desde compose!!!"

app.run(
    debug=True, 
    port=3000, 
    # Si no ponemos nada, solo escucha en 127.0.0.1
    host='0'
)

Overwriting compose_flask/code/app.py


In [56]:
%%writefile compose_flask/Dockerfile

FROM python
RUN pip install Flask
VOLUME /code
CMD ["python", "/code/app.py"]

Overwriting compose_flask/Dockerfile


In [57]:
%%writefile compose_flask/docker-compose.yml
version: '3'
    
services:
    web:
        build:
            context: .
        ports:
            - 3000:3000
        volumes:
            - ./code:/code

Overwriting compose_flask/docker-compose.yml


In [58]:
!cd compose_flask; docker-compose up

Starting composeflask_web_1 ... 
Starting composeflask_web_1
Attaching to composeflask_web_12mdone
web_1  |  * Running on http://0:3000/ (Press CTRL+C to quit)
web_1  |  * Restarting with stat
web_1  |  * Debugger is active!
web_1  |  * Debugger PIN: 283-805-230
web_1  | 172.28.0.1 - - [20/Sep/2017 15:10:49] "GET / HTTP/1.1" 200 -
web_1  | 172.28.0.1 - - [20/Sep/2017 15:10:49] "GET /favicon.ico HTTP/1.1" 404 -
^C
ERROR: Aborting.


In [60]:
!docker images

REPOSITORY                                 TAG                 IMAGE ID            CREATED             SIZE
composeflask_web                           latest              07b87b6fdd21        10 minutes ago      698MB
pascal                                     latest              3128b2d26455        18 minutes ago      1.02GB
mysql                                      5.7                 b4e78b89bcf3        31 hours ago        412MB
miservicio                                 latest              e63b9519b355        40 hours ago        683MB
<none>                                     <none>              8b0be2761d07        40 hours ago        683MB
<none>                                     <none>              f57e86b13b90        40 hours ago        683MB
ejemlplo                                   latest              b3da50436eb1        42 hours ago        683MB
wordpress                                  latest              2c69ebee76a8        5 days ago          408MB
www.eip.unp.edu.ar:

## ¿Qué sucede si editamos el código?

In [ ]:
!open compose_moodle/ctest_modeoctest_modeoctest_modeoctest_mode